In [85]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import pandas as pd 
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import  hstack
from astropy.table import  vstack
from astropy.table import unique
from astropy.table import Column
from astropy.table import join
import celerite
import imp
mod = imp.load_source('modules.py', '../code/modules.py')
from matplotlib import rcParams 
rcParams['ytick.labelsize'] = 15
rcParams['xtick.labelsize'] = 15
rcParams['axes.labelsize'] = 20
rcParams['axes.linewidth'] = 2
rcParams['font.size'] = 15
rcParams['axes.titlesize'] = 18

In [86]:
outDir = '../dp2/Fig2_simLC_SDSS-PS1/'
simLcs =  os.listdir(outDir)


In [87]:


def calc_Fig2_data(lcname):
    outDir = '../dp2/Fig2_simLC_SDSS-PS1/'
    simulated = np.load(outDir + lcname).item()

    # do just the SDSS-PS1 part first 
    sig_lims , tau_lims = [0.02,0.7], [1,5000]


    tau_min = simulated[0]['tau_in']
    tau_max = simulated[99]['tau_in']

    tau_lims = [tau_min-1, tau_max+100]
    sig_lims = [0.02, 0.7] # since sigma_in  is 0.14 for all LCs...
    sig_in = 0.14
    tau_in = 100 # starting values ... 
    prior = 'Jeff1'  # 1/sigma * 1/tau  
    t = simulated['mjd']
    yerr = simulated['y_err']

    irange = 100
    jrange = 100 

    results = { 'sigmaMAP' : np.zeros(irange*jrange, dtype=float),
               'sigmaEXP' : np.zeros(irange*jrange, dtype=float),
               'tauMAP' : np.zeros(irange*jrange, dtype=float),
               'tauEXP' : np.zeros(irange*jrange, dtype=float),
               'rhoIN' : np.zeros(irange*jrange, dtype=float),
               'tauIN' : np.zeros(irange*jrange, dtype=float),
              }

    k = 0 
    for i in range(irange):  # rho grid 
        if i % 10 == 0 : print(lcname, i )
        for j in range(jrange) :  # j-th realization .. 
            #if j % 10 == 0 :  print(lcname, i , j )

            y = simulated[i][j] # the observed magnitude 
            # find the Celerite expectation value and MAP 
            # find the MAP estimate 
            sigmaMAP, tauMAP,gp = mod.find_celerite_MAP(t ,y,yerr, 
                                      sig_in, tau_in, prior=prior,set_bounds=True, 
                                      sig_lims=sig_lims, tau_lims=tau_lims ,
                                                        verbose=False)
            # expectation from grid  
            Ngrid = 60  ; scale = 'log'
            sigma_grid, tau_grid = mod.make_grid(scale,sig_lims, tau_lims,Ngrid) 
            logP = mod.evaluate_logP(sigma_grid, tau_grid, y, gp, prior, 'celerite')

            # find the expectation value 
            sigmaEXP, tauEXP = mod.find_expectation_value(logP, sigma_grid,  tau_grid)

            # store the results 
            results['sigmaMAP'][k] = sigmaMAP
            results['sigmaEXP'][k] = sigmaEXP
            results['tauMAP'][k] = tauMAP
            results['tauEXP'][k] = tauEXP
            results['rhoIN'][k] = simulated[i]['rho_in']
            results['tauIN'][k] = simulated[i]['tau_in']

            # update the counter 
            k += 1 
    print('Done')

    # Store the results ...
    resDir = '../dp2/Fig2_results_SDSS-PS1/'

    np.save(resDir + lcname[:-4] , results , )
    print('Saved all fitted data for Fig2 for that set of cadence and errors for SDSS-PS1')
    print('  to  ',resDir + lcname[:-4]+'.npy') 

    return 

In [ ]:
# simulated LCs - 10000 per real SDSS-PS1 cadence / errors ... 
outDir = '../dp2/Fig2_simLC_SDSS-PS1/'
simLcs =  os.listdir(outDir)

# exclude those lcs that 
# have already been calculated 
resDir = '../dp2/Fig2_results_SDSS-PS1/'


select_not_done = ~np.in1d(simLcs,  os.listdir(resDir))
print(np.sum(select_not_done), '/', len(select_not_done), ' are not yet calculated')
# exclude "catastrophic failures"...
real = Table.read('190409_Celerite_real_Jeff1.txt', format='ascii')
lcname_npy = np.array([lc[:-4]+'.npy' for lc in real['lcname'].data.data])
lcname_good = lcname_npy[np.log10(real['tauEXP_sdss-ps1']) > 1.5 ]
select_not_catastrophic = np.in1d(simLcs, lcname_good)
print(np.sum(select_not_catastrophic), '/', len(select_not_catastrophic), 
      ' are not failures (log10(tauEXP) > 1.5 ) ')

simLcs_to_do = np.array(simLcs)[select_not_done & select_not_catastrophic]
print('In total, we are processing ', len(simLcs_to_do), ' of ',  len(simLcs))
for ith in range(len(simLcs_to_do)):
    print(ith,'/',len(simLcs_to_do))
    lcname = simLcs_to_do[ith]
    calc_Fig2_data(lcname)

43 / 100  are not yet calculated


/Users/chris/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log10
  from ipykernel import kernelapp as app


69 / 100  are not failures (log10(tauEXP) > 1.5 ) 
In total, we are processing  13  of  100
0 / 13
132784.npy 0
132784.npy 10
132784.npy 20
132784.npy 30
132784.npy 40
132784.npy 50
132784.npy 60
132784.npy 70
132784.npy 80
132784.npy 90
Done
Saved all fitted data for Fig2 for that set of cadence and errors for SDSS-PS1
  to   ../dp2/Fig2_results_SDSS-PS1/132784.npy
1 / 13
1141048.npy 0
1141048.npy 10
1141048.npy 20
1141048.npy 30
1141048.npy 40
1141048.npy 50
1141048.npy 60
1141048.npy 70
1141048.npy 80
1141048.npy 90
Done
Saved all fitted data for Fig2 for that set of cadence and errors for SDSS-PS1
  to   ../dp2/Fig2_results_SDSS-PS1/1141048.npy
2 / 13
1089624.npy 0
1089624.npy 10
1089624.npy 20
1089624.npy 30
1089624.npy 40
1089624.npy 50
1089624.npy 60
1089624.npy 70
1089624.npy 80
1089624.npy 90
Done
Saved all fitted data for Fig2 for that set of cadence and errors for SDSS-PS1
  to   ../dp2/Fig2_results_SDSS-PS1/1089624.npy
3 / 13
2565881.npy 0
2565881.npy 10
2565881.npy 20
2565

In [82]:
lcname

'1025069.npy'